In [118]:
import pandas as pd
import os

Importing and combining flight data sources and aircraft data source

In [119]:
#appending the monthly flight data sources into a single dataframe(takes approx 1 min to run) 
csv_files = [file for file in os.listdir("Monthly Data Sources/Flight") ]
data_frames = []
for file in csv_files:
     data_frames.append(pd.read_csv(os.path.join("Monthly Data Sources/Flight", file)))
flights_db = pd.concat(data_frames, ignore_index=True)


In [120]:
aircrafts = pd.read_csv("Monthly Data Sources/Aircraft/Aircraft.csv")

C:\Users\jenso\AppData\Local\Temp\ipykernel_19200\289985414.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  aircrafts = pd.read_csv("Monthly Data Sources/Aircraft/Aircraft.csv")


In [121]:
#creating lookup table source for the flight dataset
airline_look_up = pd.read_csv("Look Up Tables/L_AIRLINE_ID.csv")
airport_look_up = pd.read_csv("Look Up Tables/L_AIRPORT_ID.csv")
cancellation_look_up = pd.read_csv("Look Up Tables/L_CANCELLATION.csv")
country_look_up = pd.read_csv("Look Up Tables/L_WORLD_AREA_CODES.csv")

Merging the dataset together

In [122]:
flights_db.dtypes

YEAR                       int64
MONTH                      int64
DAY_OF_MONTH               int64
FL_DATE                   object
OP_CARRIER_AIRLINE_ID      int64
TAIL_NUM                  object
OP_CARRIER_FL_NUM          int64
ORIGIN_AIRPORT_ID          int64
ORIGIN_AIRPORT_SEQ_ID      int64
ORIGIN_CITY_MARKET_ID      int64
ORIGIN                    object
ORIGIN_CITY_NAME          object
ORIGIN_WAC                 int64
DEST_AIRPORT_ID            int64
DEST_AIRPORT_SEQ_ID        int64
DEST_CITY_MARKET_ID        int64
DEST                      object
DEST_CITY_NAME            object
DEST_WAC                   int64
CRS_DEP_TIME               int64
DEP_TIME                 float64
DEP_DELAY                float64
CRS_ARR_TIME               int64
ARR_TIME                 float64
ARR_DELAY                float64
CANCELLED                float64
CANCELLATION_CODE         object
DIVERTED                 float64
CRS_ELAPSED_TIME         float64
ACTUAL_ELAPSED_TIME      float64
AIR_TIME  

Creating the Fact Table and transforming the fact table

In [123]:
fact_table = flights_db[["CANCELLATION_CODE", 
                      "OP_CARRIER_FL_NUM", 
                      "FL_DATE", 
                      "OP_CARRIER_AIRLINE_ID", 
                      "TAIL_NUM", 
                      "ORIGIN_AIRPORT_ID", 
                      "DEST_AIRPORT_ID", 
                      "CARRIER_DELAY", 
                      "WEATHER_DELAY", 
                      "NAS_DELAY", 
                      "SECURITY_DELAY", 
                      "LATE_AIRCRAFT_DELAY"]]

In [124]:
fact_table.isnull().sum()

CANCELLATION_CODE        12739711
OP_CARRIER_FL_NUM               0
FL_DATE                         0
OP_CARRIER_AIRLINE_ID           0
TAIL_NUM                    49775
ORIGIN_AIRPORT_ID               0
DEST_AIRPORT_ID                 0
CARRIER_DELAY            10332287
WEATHER_DELAY            10332287
NAS_DELAY                10332287
SECURITY_DELAY           10332287
LATE_AIRCRAFT_DELAY      10332287
dtype: int64

In [125]:
#Handling null value
fact_table["CARRIER_DELAY"] = fact_table["CARRIER_DELAY"].fillna(0)
fact_table["WEATHER_DELAY"] = fact_table["WEATHER_DELAY"].fillna(0)
fact_table["SECURITY_DELAY"] = fact_table["SECURITY_DELAY"].fillna(0)
fact_table["NAS_DELAY"] = fact_table["NAS_DELAY"].fillna(0)
fact_table["LATE_AIRCRAFT_DELAY"] = fact_table["LATE_AIRCRAFT_DELAY"].fillna(0)

fact_table["TAIL_NUM"] = fact_table["TAIL_NUM"].replace("nan", "unknown")
fact_table["CANCELLATION_CODE"] = fact_table["CANCELLATION_CODE"].replace("nan", "E")


C:\Users\jenso\AppData\Local\Temp\ipykernel_19200\3257795551.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_table["CARRIER_DELAY"] = fact_table["CARRIER_DELAY"].fillna(0)
C:\Users\jenso\AppData\Local\Temp\ipykernel_19200\3257795551.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_table["WEATHER_DELAY"] = fact_table["WEATHER_DELAY"].fillna(0)
C:\Users\jenso\AppData\Local\Temp\ipykernel_19200\3257795551.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [126]:
fact_table.isnull().sum()

CANCELLATION_CODE        12739711
OP_CARRIER_FL_NUM               0
FL_DATE                         0
OP_CARRIER_AIRLINE_ID           0
TAIL_NUM                    49775
ORIGIN_AIRPORT_ID               0
DEST_AIRPORT_ID                 0
CARRIER_DELAY                   0
WEATHER_DELAY                   0
NAS_DELAY                       0
SECURITY_DELAY                  0
LATE_AIRCRAFT_DELAY             0
dtype: int64

In [127]:
fact_table["TAIL_NUM"] = fact_table["TAIL_NUM"].str.replace("N",'')

C:\Users\jenso\AppData\Local\Temp\ipykernel_19200\2967892618.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_table["TAIL_NUM"] = fact_table["TAIL_NUM"].str.replace("N",'')


In [128]:
fact_table

,CANCELLATION_CODE,OP_CARRIER_FL_NUM,FL_DATE,OP_CARRIER_AIRLINE_ID,TAIL_NUM,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,B,1275,1/1/2022 12:00:00 AM,19393,NaN,13796,11292,0.0,0.0,0.0,0.0,0.0
1,B,1336,1/1/2022 12:00:00 AM,19393,NaN,10693,13204,0.0,0.0,0.0,0.0,0.0
2,B,1414,1/1/2022 12:00:00 AM,19393,NaN,11292,13232,0.0,0.0,0.0,0.0,0.0
3,B,1428,1/1/2022 12:00:00 AM,19393,NaN,12953,13232,0.0,0.0,0.0,0.0,0.0
4,B,1513,1/1/2022 12:00:00 AM,19393,NaN,13198,15304,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
13006625,NaN,5853,9/30/2022 12:00:00 AM,20452,880RW,11278,13871,0.0,0.0,0.0,0.0,0.0
13006626,NaN,5853,9/30/2022 12:00:00 AM,20452,880RW,13871,11278,0.0,0.0,0.0,0.0,0.0
13006627,NaN,3418,9/30/2022 12:00:00 AM,20452,979RP,11193,12264,0.0,0.0,0.0,0.0,0.0
13006628,NaN,3486,9/30/2022 12:00:00 AM,20452,979RP,12339,13930,0.0,0.0,0.0,0.0,0.0


Creating Cancellation Dimension

In [129]:
cancellation = flights_db["CANCELLATION_CODE"].drop_duplicates()


In [130]:
cancellation = cancellation.fillna("E")

In [131]:

cancellation = pd.merge(cancellation, cancellation_look_up, left_on="CANCELLATION_CODE", right_on="Code")

In [132]:
cancellation = cancellation[["CANCELLATION_CODE", "Description"]]

In [133]:
cancellation

,CANCELLATION_CODE,Description
0,B,Weather
1,A,Carrier
2,E,No Cancellation
3,C,National Air System
4,D,Security


Creating Aircraft Dimensions

In [134]:
aircrafts = aircrafts[["N-NUMBER","SERIAL NUMBER", "YEAR MFR", "NAME", "EXPIRATION DATE"]]

In [141]:
aircrafts.isnull().sum()

N-NUMBER           0
SERIAL NUMBER      0
YEAR MFR           0
NAME               0
EXPIRATION DATE    0
dtype: int64

Creating Flight Date Dimension

In [142]:
aircrafts.dtypes

N-NUMBER           object
SERIAL NUMBER      object
YEAR MFR           object
NAME               object
EXPIRATION DATE    object
dtype: object

In [143]:
aircrafts

,N-NUMBER,SERIAL NUMBER,YEAR MFR,NAME,EXPIRATION DATE
0,1,680-0519,2014,TENAX AEROSPACE LLC ...,20281130
1,100,5334,1940,BENE MARY D ...,20270430
2,10001,A28,1928,STOOS ROBERT A ...,20290228
3,10004,T18208245,,ETOS AIR LLC ...,20290331
4,10006,BG-72,1955,COUTCHES ROBERT HERCULES DBA ...,20280229
...,...,...,...,...,...
293460,9ZR,2228,,SOUTHERN AIRCRAFT CONSULTANCY INC TRUSTEE ...,20300831
293461,9ZS,20004,1974,COLLINS BRIAN D ...,20280831
293462,9ZT,0088,2001,VALIAIR LC TRUSTEE ...,20290531
293463,9ZU,18-7028,1959,FOWLER RONALD W ...,20271130


In [139]:
flight_date = flights_db[["FL_DATE", "YEAR", "MONTH", "DAY_OF_MONTH"]].drop_duplicates()

In [140]:
flight_date.isnull().sum()

FL_DATE         0
YEAR            0
MONTH           0
DAY_OF_MONTH    0
dtype: int64

In [144]:
flight_date.dtypes

FL_DATE         object
YEAR             int64
MONTH            int64
DAY_OF_MONTH     int64
dtype: object

Creating